# How to match? 

## 0. Import libraries 

In [1]:
import pandas as pd
import warnings
import numpy as np
from lightautoml.addons.matcher import Matcher

warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [2]:
# Simulating dataset with known effect size
num_users = 10000
num_months = 12

signup_months = np.random.choice(np.arange(1, num_months), num_users) * np.random.randint(0,2, size=num_users) # signup_months == 0 means customer did not sign up
df = pd.DataFrame({
    'user_id': np.repeat(np.arange(num_users), num_months),
    'signup_month': np.repeat(signup_months, num_months), # signup month == 0 means customer did not sign up
    'month': np.tile(np.arange(1, num_months+1), num_users), # months are from 1 to 12
    'spend': np.random.poisson(500, num_users*num_months) #np.random.beta(a=2, b=5, size=num_users * num_months)*1000 # centered at 500
})
# A customer is in the treatment group if and only if they signed up
df["treat"] = df["signup_month"]>0
# Simulating an effect of month (monotonically decreasing--customers buy less later in the year)
df["spend"] = df["spend"] - df["month"]*10
# Simulating a simple treatment effect of 100
after_signup = (df["signup_month"] < df["month"]) & (df["treat"])
df.loc[after_signup,"spend"] = df[after_signup]["spend"] + 100
df.head()

,user_id,signup_month,month,spend,treat
0,0,7,1,497,True
1,0,7,2,521,True
2,0,7,3,469,True
3,0,7,4,499,True
4,0,7,5,430,True


In [3]:
# Setting the signup month (for ease of analysis)
i = 3
df_i_signupmonth = (
    df[df.signup_month.isin([0, i])]
    .groupby(["user_id", "signup_month", "treat"])
    .apply(
        lambda x: pd.Series(
            {
                "pre_spends": x.loc[x.month < i, "spend"].mean(),
                "post_spends": x.loc[x.month > i, "spend"].mean(),
            }
        )
    )
    .reset_index()
)
df_i_signupmonth

,user_id,signup_month,treat,pre_spends,post_spends
0,2,0,False,485.5,428.777778
1,4,0,False,490.0,426.777778
2,5,0,False,476.0,427.555556
3,10,0,False,471.5,413.333333
4,11,0,False,454.5,404.444444
...,...,...,...,...,...
5477,9990,0,False,473.0,435.333333
5478,9992,0,False,446.5,411.888889
5479,9994,0,False,488.0,423.444444
5480,9996,0,False,501.0,422.333333


In [4]:
# Additional category features
gender = np.random.choice(a=[0,1], size=df_i_signupmonth.user_id.nunique())
age = np.random.choice(a=range(18, 70), size=df_i_signupmonth.user_id.nunique())
industry = np.random.choice(a=range(1, 3), size=df_i_signupmonth.user_id.nunique())
df_i_signupmonth['age'] = age
df_i_signupmonth['is_male'] =  gender
df_i_signupmonth['industry'] =  industry
df_i_signupmonth['industry'] = df_i_signupmonth['industry'].astype('str')
df_i_signupmonth['treat'] = df_i_signupmonth['treat'].astype(int)
df_i_signupmonth.head()

,user_id,signup_month,treat,pre_spends,post_spends,age,is_male,industry
0,2,0,0,485.5,428.777778,47,0,1
1,4,0,0,490.0,426.777778,45,1,2
2,5,0,0,476.0,427.555556,62,0,1
3,10,0,0,471.5,413.333333,51,0,1
4,11,0,0,454.5,404.444444,25,1,1


In [5]:
df_i_signupmonth.columns

Index(['user_id', 'signup_month', 'treat', 'pre_spends', 'post_spends', 'age',
       'is_male', 'industry'],
      dtype='object')

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [6]:
info_col = ['user_id', 'signup_month']

outcome = 'post_spends'
treatment = 'treat'

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [7]:
# Standard model with base parameters
model = Matcher(df=df_i_signupmonth, outcome=outcome, treatment=treatment, info_col=info_col)
results, quality_results = model.estimate()

Finding index
Finding index


### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [8]:
# group_col is used for strict comparison of categorical features.  
# In our case there is only one attribute  
# If there are several such attributes, you should make one of them and use it

group_col = "industry"

In [9]:
model = Matcher(df=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)
results, quality_results = model.estimate()

Finding index
Finding index
Finding index
Finding index


### 2.3 Matching but you don't know which features are better to use  
This is the easiest way to initialize and calculate metrics on a Matching task  
Use this method when you want to select the most important features and make matching based on them. 

In [10]:
model = Matcher(df=df_i_signupmonth, outcome=outcome, treatment=treatment, 
                info_col=info_col, group_col=group_col)

In [11]:
features_importance = model.lama_feature_select()
features_importance

,Feature,Importance
0,pre_spends,36304.409912
1,age,20823.709961
2,is_male,3080.989990


In [12]:
features = features_importance['Feature'].to_list()

In [13]:
# You can use both variant that you like. So you can just list the features to be matched

#results, quality_results = model.estimate(features=features_importance[:3])
results, quality_results = model.estimate(features=features[:3])

Finding index
Finding index
Finding index
Finding index


In [14]:
model.matcher.df_matched

,index,user_id,signup_month,pre_spends,age,is_male,industry,index_matched,user_id_matched,signup_month_matched,pre_spends_matched,age_matched,is_male_matched,industry_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,2299,4156,3,495.5,38,1,1,5141,9357,0,497.0,45,0,1,428.777778,530.666667,530.692975,0,1
1,4380,7920,3,472.5,26,0,1,2984,5357,0,467.0,24,1,1,423.555556,520.777778,520.643947,0,1
2,2270,4104,3,456.5,41,0,1,2799,5017,0,456.0,37,1,1,414.777778,531.888889,531.867511,0,1
3,2340,4210,3,451.0,18,0,1,2837,5084,0,449.0,20,0,1,418.777778,517.111111,517.180487,0,1
4,2360,4241,3,483.0,46,1,1,4284,7741,0,486.5,55,0,1,410.111111,520.333333,520.157086,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5477,2510,4500,0,475.0,27,0,2,2133,3866,3,466.5,28,0,2,511.222222,412.444444,411.793345,1,0
5478,2512,4505,0,485.0,46,1,2,3636,6530,3,493.5,48,1,2,529.000000,418.333333,418.492023,1,0
5479,2514,4511,0,498.5,60,0,2,1908,3448,3,488.0,51,0,2,523.444444,409.222222,409.550086,1,0
5480,1844,3339,0,517.5,18,0,2,3543,6356,3,455.5,69,1,2,521.000000,407.555556,407.336581,1,0


## 3. Results  
### 3.1 ATE, ATT, ATC

In [15]:
# model.matcher.results
results

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,100.097572,0.649687,0.0,98.824186,101.370959
ATC,100.182530,0.671551,0.0,98.866290,101.498770
ATT,99.156677,0.735807,0.0,97.714495,100.598858


### 3.2 SMD, PSI, KS-test, repeats

In [16]:
# matching quality result - SMD
model.quality_result['smd']

,match_control_to_treat,match_treat_to_control
pre_spends,0.086449,0.014896
age,0.007773,0.007438
is_male,0.017622,0.015127


In [17]:
# matching quality result - PSI
model.quality_result['psi']

,column,anomaly_score,check_result,column,anomaly_score,check_result
0,pre_spends_treated,0.07,OK,pre_spends_untreated,0.00,OK
1,age_treated,0.15,OK,age_untreated,0.07,OK
2,is_male_treated,0.00,OK,is_male_untreated,0.00,OK
3,post_spends_treated,16.11,NOK,post_spends_untreated,8.27,NOK


In [18]:
# matching quality result - KS-test

model.quality_result['ks_test']

,match_control_to_treat,match_treat_to_control
pre_spends,0.058908,2.229286e-01
age,0.156742,4.686527e-07
is_male,1.000000,9.987747e-01


In [19]:
# matching quality result - repeats
model.quality_result['repeats']

{'match_control_to_treat': 0.89, 'match_treat_to_control': 0.08}

### 3.3 Validation
validate result with placebo treatment  
our new effect size is close to zero (it should be)

In [20]:
 
model.validate_result()
 

Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index
Finding index


{'post_spends': -1.4320949937395813}

In [21]:
model.matcher.df_matched

,index,user_id,signup_month,pre_spends,age,is_male,industry,index_matched,user_id_matched,signup_month_matched,pre_spends_matched,age_matched,is_male_matched,industry_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,2282,4125,0,472.0,23,1,1,2058,3724,0,457.5,60,1,1,428.777778,419.000000,417.767251,0,1
1,2292,4146,0,490.0,49,0,1,699,1304,0,490.0,29,1,1,423.555556,517.777778,516.082198,0,1
2,2293,4147,0,484.5,39,1,1,3248,5837,0,477.0,57,0,1,414.777778,411.222222,412.035568,0,1
3,4400,7966,0,489.0,51,1,1,1701,3084,0,490.5,58,0,1,410.111111,419.111111,420.617851,0,1
4,4394,7950,0,497.0,67,0,1,2445,4403,0,511.0,29,0,1,420.333333,418.888889,414.799991,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5477,2510,4500,0,475.0,27,0,2,4938,8955,0,478.0,56,0,2,419.666667,429.000000,428.781057,1,0
5478,2512,4505,0,485.0,46,1,2,1996,3604,3,485.5,44,1,2,529.000000,417.888889,417.788771,1,0
5479,2513,4509,3,489.5,44,0,2,2677,4814,0,478.5,36,0,2,432.888889,414.555556,414.382788,1,0
5480,1844,3339,0,517.5,18,0,2,2190,3955,0,448.5,33,0,2,433.555556,411.777778,411.600957,1,0
